DEPENDENCIAS

In [1]:
import pandas as pd

LEEMOS LOS DATOS

In [2]:
datos = pd.read_excel("./inputs/Gerd_Eurostat.xlsx")
datos.head()

,"sectfund,unit,geo\time",2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,"ABR,PC_TOT,AT",20 e,19.4,18 e,18.4,17.9,16.4 e,16.8,16.1 e,16.9,15.7 e,16.6,15.2 ep
1,"ABR,PC_TOT,BA",:,:,:,:,:,:,:,:,:,48.7,53.9,:
2,"ABR,PC_TOT,BE",12.9,12.3,12.4,13.3,13,12.3,12.1,13.3,13,13,13.2,:
3,"ABR,PC_TOT,BG",5.8,5.5,7.6,6.5,7.6,6.8,8.4,39.6,43.9,46.3,48.3,:
4,"ABR,PC_TOT,CH",:,5.2,:,:,:,6,:,:,:,12.1,:,:
